# M-estimation: a worked example with connections to maximum likelihood estimation 

Python (XXX 2023/01/16)

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy as sp
from scipy.optimize import minimize, approx_fprime, newton
import delicatessen as deli
from delicatessen import MEstimator
from delicatessen.estimating_equations import ee_regression
from delicatessen.utilities import inverse_logit

print("versions")
print("--------------------")
print("NumPy:       ", np.__version__)
print("SciPy:       ", sp.__version__)
print("pandas:      ", pd.__version__)
print("statsmodels: ", sm.__version__)
print("Delicatessen:", deli.__version__)

versions
--------------------
NumPy:        1.22.2
SciPy:        1.9.2
pandas:       1.4.1
statsmodels:  0.13.2
Delicatessen: 1.1


### Loading data
Generating the corresponding data set from Table 1

In [2]:
# From Table 1
d = pd.DataFrame()
d['X'] = [0, 0, 0, 0, 1, 1, 1, 1]
d['W'] = [0, 0, 1, 1, 0, 0, 1, 1]
d['Y'] = [0, 1, 0, 1, 0, 1, 0, 1]
d['n'] = [496, 74, 113, 25, 85, 15, 15, 3]
d['intercept'] = 1

# Expanding rows by n
d = pd.DataFrame(np.repeat(d.values, d['n'], axis=0), columns=d.columns)
d = d[['intercept', 'X', 'W', 'Y']].copy()

### Maximum Likelihood Estimation (MLE) by-hand
First, we will do the MLE by-hand using `scipy.optimize` with the Nelder-Mead algorithm.

In [3]:
def loglikelihood(beta):
    # Transform log-odds to probability
    prob = inverse_logit(np.dot(X, beta))
    # Individual contributions to log-likelihood
    likelihood_i = y*np.log(prob) + (1-y)*np.log(1-prob)
    # Sum of all individual contributions (negative since minimize)
    return -np.sum(likelihood_i)

In [4]:
# Formatting data for likelihood optimization
X = np.asarray(d[['intercept', 'X', 'W']])
y = np.asarray(d['Y'])

# Calling SciPy for optimization procedure
mle = minimize(loglikelihood, [-2, 0, 0], method='Nelder-Mead')
mle.x

array([-1.89450473,  0.11877963,  0.36047201])

### MLE with `statsmodels`
Using `statsmodels` version of the generalized linear model to estimate the logistic model parameters

In [5]:
f = sm.families.Binomial()
fm = smf.glm("Y ~ X + W", d, family=f).fit()
fm.params  # Point estimates

Intercept   -1.894501
X            0.118735
W            0.360511
dtype: float64

In [6]:
fm.bse**2  # Variance (Hessian-based)

Intercept    0.014960
X            0.077648
W            0.056605
dtype: float64

### M-estimation by-hand

In [7]:
def ee_logistic(theta):
    # Estimating equation for the logistic model
    beta = np.asarray(theta)[:, None]
    n = d.shape[0]

    # Looping through each observation
    est_vals = []
    for i in range(n):
        v_i = (y[i] - inverse_logit(np.dot(X[i], beta)))*X[i]
        est_vals.append(v_i)

    return np.asarray(est_vals).T


def sum_ee(theta):
    # Function to sum the previous estimating equation over all i's
    stacked_equations = np.asarray(ee_logistic(theta))  # Returning stacked equation
    vals = ()                                           # Create empty tuple
    for i in stacked_equations:                         # Go through each individual theta
        vals += (np.sum(i), )                           # Add the theta sum to the tuple of thetas

    # Return the calculated values of theta
    return vals


def solve_m_estimator(stacked_equations, init):
    # Wrapper function for SciPy root-finding 
    psi = newton(stacked_equations,    # stacked equations to solve (should be written as sums)
                 x0=np.asarray(init),  # initial values for solver
                 maxiter=2000,         # Increasing iterations
                 disp=True)            # Option to raise RuntimeError if doesn't converge
    return psi

In [8]:
# Solving the estimating equations for beta
theta = solve_m_estimator(stacked_equations=sum_ee,
                          init=[0, 0, 0]
                          )
print(theta)

[-1.89449987  0.11873458  0.36051038]


In [9]:
# Bread matrix computation
bread = -approx_fprime(theta, sum_ee)
bread_invert = np.linalg.inv(bread)

# Meat matrix computation
x = np.asarray(ee_logistic(theta=theta))
meat = np.dot(x, x.T)

# Sandwich variance
sandwich = np.dot(np.dot(bread_invert, meat), bread_invert.T)
sandwich_var = np.diag(sandwich)
sandwich_var

array([0.01484041, 0.07772034, 0.05652968])

### M-estimation with `delicatessen`

In [10]:
def psi(theta):
    return ee_regression(theta=theta, X=X, y=y, 
                         model='logistic')

In [11]:
mestr = MEstimator(psi, init=[0, 0, 0])
mestr.estimate(solver='lm')

# Point estimates
mestr.theta

array([-1.89450082,  0.11873535,  0.36051133])

In [12]:
# Sandwich variance
np.diag(mestr.variance)

array([0.01484041, 0.0777204 , 0.05652963])

In [13]:
# Hessian-based variance
np.diag(np.linalg.inv(mestr.bread) / d.shape[0])

array([0.01496049, 0.07764846, 0.05660454])

In [14]:
# Outer-product variance
np.diag(np.linalg.inv(mestr.meat) / d.shape[0])

array([0.01508328, 0.07761366, 0.05670628])

### Table 2 Replication

In [15]:
t2 = pd.DataFrame()
t2['MLE-1'] = fm.params
t2['MLE-2'] = mle.x
t2['Root-1'] = theta
t2['Root-2'] = mestr.theta
t2['Hessian'] = fm.bse**2
t2['OuterProd'] = np.diag(np.linalg.inv(mestr.meat) / d.shape[0])
t2['Sandwich-1'] = sandwich_var
t2['Sandwich-2'] = np.diag(mestr.variance)

t2

,MLE-1,MLE-2,Root-1,Root-2,Hessian,OuterProd,Sandwich-1,Sandwich-2
Intercept,-1.894501,-1.894505,-1.894500,-1.894501,0.014960,0.015083,0.01484,0.01484
X,0.118735,0.118780,0.118735,0.118735,0.077648,0.077614,0.07772,0.07772
W,0.360511,0.360472,0.360510,0.360511,0.056605,0.056706,0.05653,0.05653
